In [1]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time



from tqdm import tqdm_notebook



In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import matplotlib
import matplotlib.pyplot as plt  # 파이플롯 사용
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')  # 한글코드를 더 선명하게 해주는 조치, 레티나 설정
matplotlib.rc('font', family='AppleGothic') # 폰트 설정
matplotlib.rc('axes', unicode_minus=False) # 마이너스 폰트가 깨지는 경우가 있으므로 조치

import seaborn as sns
sns.set_style('whitegrid')

In [3]:
df = pd.read_csv("kbo_stat.csv", engine='python')


In [4]:
df.insert(3, 'war', -99)
#df.insert(4, 'position', np.nan)


In [ ]:
from tqdm import tqdm_notebook

chromedriver = '/Users/taewoongkong/workspace/selenium_learning/chromedriver'
main_driver = webdriver.Chrome(chromedriver)

time.sleep(0.3)

for year in tqdm_notebook(range(2002, 2020)):
    print(f"http://www.statiz.co.kr/stat_at.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=2013&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=50&si=&cn=")
    main_driver.get(f"http://www.statiz.co.kr/stat_at.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=2013&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=2&sn=50&si=&cn=")
    time.sleep(3)
    
    for num in range(3, 53):
        temp_name = main_driver.find_element_by_css_selector(f"#fixcol > table > tbody > tr:nth-child({num}) > td:nth-child(2) > a").text
        temp_war = main_driver.find_element_by_css_selector(f"#fixcol > table > tbody > tr:nth-child({num}) > td:nth-child(4) > font > span").text
        #temp_position = main_driver.find_element_by_css_selector(f"#fixcol > table > tbody > tr:nth-child({num}) > td:nth-child(3) > span > span:nth-child(3)").text
        print(temp_name)
        print(temp_war)
        #print(temp_position)


        df.loc[(df['year'] == year) & (df['player_name'] == temp_name), 'war'] = temp_war
        #df.loc[(df['year'] == year) & (df['player_name'] == temp_name), 'position'] = temp_position

    time.sleep(3)

main_driver.quit()


In [10]:
df['war'] = df['war'].astype(float)

In [19]:
df_war = df.loc[~(df['war'] == -99)]

In [15]:
df_war.to_csv("df_war.csv", mode='a', header=True)

In [20]:
cols = df_war.columns.tolist()
cols = cols[:3] + cols[4:] + [cols[3]]
df_war = df_war[cols]

In [22]:
X_df = df_war.drop(['player_name', 'team'], axis=1).iloc[:, :-1]
y_df = pd.merge(df_war.iloc[:, 0], df_war.iloc[:, -1], left_on=None, right_on=None, left_index=True, right_index=True)



In [61]:
X_train = X_df.loc[X_df['year'] < 2019].iloc[:, 1:]
X_test = X_df.loc[X_df['year'] == 2019].iloc[:, 1:]

y_train = y_df.loc[X_df['year'] < 2019].iloc[:, -1]
y_test = y_df.loc[X_df['year'] == 2019].iloc[:, -1]



In [46]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

# 적용 모델이 여럿일 때 한번에 굴리기
def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [62]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso


lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)

ridge_reg = Ridge(alpha = 1.0)  # 1.0 alpha가 디폴트임
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

LinearRegression RMSE: 0.67
Ridge RMSE: 0.67
Lasso RMSE: 0.724


[0.6704361995684213, 0.6704312047385093, 0.7243885325818251]

In [63]:
# 예시 모델 릿지
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

pred = ridge_reg.predict(X_test)

print('알스퀘어드(Variance score) : {0:.3f}'.format(r2_score(y_test, pred)))

알스퀘어드(Variance score) : 0.742


In [64]:
print('MAE : {0:.3f}'.format(mean_absolute_error(y_test, pred)))

MAE : 0.490


In [ ]:
kb